<div class="alert alert-block alert-success">
    <h1 align="center">Machine Learning in Python</h1>
    <h3 align="center">Artificial Neural Network</h3>
    <h4 align="center"><a href="https://github.com/AliBinary">Ali Ghanbari</a></h5>
</div>

### Using PyTorch - Artificial Neural Network

## Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

### Check PyTorch version and GPU

In [ ]:
print("PyTorch version:", torch.__version__)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

## 2️⃣ Load Dataset

In [ ]:
dataset = pd.read_csv('Churn_Modelling.csv')

### Features and labels

In [ ]:
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

## 3️⃣ Encode Categorical Data
### Gender

In [ ]:
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

## Geography (OneHot)

In [ ]:
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

## 4️⃣ Split Dataset

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

## 5️⃣ Feature Scaling

In [ ]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

### Convert to PyTorch tensors

In [ ]:
X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1).to(device)
y_test = torch.tensor(y_test, dtype=torch.float32).view(-1, 1).to(device)

## 6️⃣ Build ANN Model

In [ ]:
class ANNModel(nn.Module):
    def __init__(self, input_dim):
        super(ANNModel, self).__init__()
        self.layer1 = nn.Linear(input_dim, 6)
        self.layer2 = nn.Linear(6, 6)
        self.output = nn.Linear(6, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.sigmoid(self.output(x))
        return x

input_dim = X_train.shape[1]
model = ANNModel(input_dim).to(device)

## 7️⃣ Compile Model

In [ ]:
criterion = nn.BCELoss()           # Binary Cross Entropy
optimizer = optim.Adam(model.parameters(), lr=0.001)

## 8️⃣ Train Model

In [ ]:
epochs = 100
batch_size = 32

for epoch in range(epochs):
    permutation = torch.randperm(X_train.size()[0])
    
    for i in range(0, X_train.size()[0], batch_size):
        indices = permutation[i:i+batch_size]
        batch_X, batch_y = X_train[indices], y_train[indices]
        
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

## 9️⃣ Evaluate Model

In [ ]:
with torch.no_grad():
    y_pred = model(X_test)
    y_pred_class = (y_pred > 0.5).float()

# Confusion matrix manually
tp = ((y_pred_class == 1) & (y_test == 1)).sum().item()
tn = ((y_pred_class == 0) & (y_test == 0)).sum().item()
fp = ((y_pred_class == 1) & (y_test == 0)).sum().item()
fn = ((y_pred_class == 0) & (y_test == 1)).sum().item()

print("Confusion Matrix:")
print([[tn, fp],
       [fn, tp]])

accuracy = (tp + tn) / (tp + tn + fp + fn)
print("Accuracy:", accuracy)

## 1️⃣0️⃣ Predict Single Customer
## Customer Info (France, 600, Male, 40, 3, 60000, 2, Yes, Yes, 50000)

In [ ]:
new_customer = np.array([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])
new_customer_scaled = sc.transform(new_customer)
new_customer_tensor = torch.tensor(new_customer_scaled, dtype=torch.float32).to(device)

with torch.no_grad():
    prediction = model(new_customer_tensor)
    print("Will the customer leave?", prediction.item() > 0.5)